In [ ]:
import numpy as np
import pandas as pd
import os
import seaborn as sns; sns.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, BatchNormalization, Dropout, MaxPool2D
from tqdm import tqdm
import time
import random

### **Submission**

In [ ]:
class_names = ['Arjun_Rampal', 'Arshad_Warsi', 'Asin', 'Ayushmann_Khurrana', 'Bhumi_Pednekar', 'Bipasha_Basu', 'Bobby_Deol', 'Deepika_Padukone', 'Disha_Patani', 'Emraan_Hashmi', 'Esha_Gupta', 'Farhan_Akhtar', 'Govinda']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}
nb_classes = len(class_names)
IMAGE_SIZE = (160, 160)

In [ ]:
# read the input from MTCNN
data = np.load('data_bw.npz')
trainX,  trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

In [ ]:
n_train = trainy.shape[0]
n_test = testy.shape[0]

print ("Number of training examples: {}".format(n_train))
print ("Number of testing examples: {}".format(n_test))
print ("Each face is of size: {}".format(IMAGE_SIZE))

In [ ]:
import pandas as pd
fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1, 1, 1)
_, train_counts = np.unique(trainy, return_counts=True)
_, test_counts = np.unique(testy, return_counts=True)
# pd.DataFrame({'train': train_counts,'test': test_counts}, index=class_names).plot.bar(ax=ax)
# res = pd.DataFrame({'train': train_counts,'test': test_counts}, index=class_names)
# res
res = pd.concat([pd.DataFrame(train_counts, index=class_names), pd.DataFrame(test_counts, index=class_names)])
res.columns = ['count']
res['train_or_test'] = np.array([1]*13 + [0]*13)
res['train_or_test'] = np.where(res['train_or_test'] == 1, 'train', 'test')
res.index.name = 'celebrity'
res.reset_index(drop=False, inplace=True)
sns.barplot(x='celebrity', y='count', hue='train_or_test', data=res)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
# plt.savefig("./figures/train_test.png", dpi=800) 

In [ ]:
# standardization
trainX = trainX / 255
testX = testX / 255

In [ ]:
# shuffle to make the validation set random
random.seed(4012)

trainX, trainy = shuffle(trainX, trainy, random_state=4012)

In [ ]:
# display a random image
def display_random_image(class_names, images, labels):
    index = np.random.randint(images.shape[0])
    plt.figure()
    plt.imshow(images[index])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.title('Image #{} : '.format(index) + labels[index])
    plt.show()

In [ ]:
display_random_image(class_names, trainX, trainy)

In [ ]:
# display group of faces
def display_examples(class_names, images, labels):
    """
        Display 25 images from the images array with its corresponding labels
    """
    
    fig = plt.figure(figsize=(14,14))
    for i in range(25):
        index = np.random.randint(images.shape[0])
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[index], cmap=plt.cm.binary)
        # plt.imshow(images[i], cmap=plt.cm.binary)
        plt.xlabel(labels[i])
    plt.show()
display_examples(class_names, trainX, trainy)


In [ ]:
# one-hot encoding
# perform one-hot encoding on train label
from sklearn.preprocessing import OneHotEncoder 
print(trainy.shape) #a list without second shape
print(trainy.reshape(-1, 1).shape) #change to a 2d array
encoder = OneHotEncoder()
trainy_hot = encoder.fit_transform(trainy.reshape(-1,1))
trainy_hot 
# trainy_hot is a sparse matrix, convert it to an ndarray
trainy_hot.toarray()

In [ ]:
# perform one hot encoding on test label
from sklearn.preprocessing import OneHotEncoder 
encoder = OneHotEncoder()
test_labels_hot = encoder.fit_transform(testy.reshape(-1,1))
test_labels_hot 
test_labels_hot.toarray()

### **CNN fitting**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

tf.random.set_seed(4012)

# possible model 1
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (160, 160, 3)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(13, activation = "softmax"))
model.compile(optimizer='adam' , loss = "binary_crossentropy", metrics=["accuracy"])
history = model.fit(trainX, trainy_hot.toarray(), batch_size=64, epochs=50, validation_split = 0.2)


# possible model 2
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
#                       patience=3, 
#                       verbose=1, 
#                       factor=0.7, 
#                       min_lr=0.00000000001)

# model = Sequential()

# model.add(Conv2D(filters = 20, kernel_size = (5,5),padding = 'Same', 
#                  activation ='relu', input_shape = (160,160,3)))

# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(filters = 50, kernel_size = (6,6),padding = 'Same', 
#                  activation ='relu'))

# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(filters = 150, kernel_size = (5,5),padding = 'Same', 
#                  activation ='relu', input_shape = (160,160,3)))

# model.add(Flatten())
# model.add(Dense(256, activation = "relu"))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(Dense(13, activation = "softmax"))

# model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics=["accuracy"])
# history = model.fit(trainX, trainy_hot.toarray(), batch_size=32, epochs=100, validation_split = 0.2, callbacks=[learning_rate_reduction])

In [ ]:
print(model.summary())

In [ ]:
# evaluate on the test set
test_loss = model.evaluate(testX, test_labels_hot.toarray())
test_loss

In [ ]:
def plot_accuracy_loss(history):
    """
        Plot the accuracy and the loss during the training of the nn.
    """
    fig = plt.figure(figsize=(18,5))

    # Plot accuracy
    plt.subplot(121)
    plt.plot(history.history['accuracy'],'bo--', label = "accuracy")
    plt.plot(history.history['val_accuracy'], 'ro--', label = "val_accuracy")
    plt.title("train_accuracy vs val_accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()

    # Plot loss function
    plt.subplot(122)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")
    plt.subplots_adjust()
    plt.tight_layout()
    plt.legend()
    plt.show()

In [ ]:
plot_accuracy_loss(history)

In [ ]:
# define another randomly display function to visualize the result
def display_random_image2(class_names, images, labels):
    index = np.random.randint(images.shape[0])
    plt.figure()
    plt.imshow(images[index])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.title('Image #{} : '.format(index) + labels[index])
    plt.show()

# convert the prediction result (in the format of 2d array) to the name of the celebrity (string)
predictions = np.round(model.predict(testX)).astype(int)    # Vector of probabilities
pred_labels = np.squeeze(predictions) 
pred = np.array([class_names[i.argmax()] for i in pred_labels])
display_random_image2(class_names, testX, pred)

In [ ]:
predictions = np.round(model.predict(trainX)).astype(int)    # Vector of probabilities
train_labels = np.squeeze(predictions) 
train = np.array([class_names[i.argmax()] for i in train_labels])

In [ ]:
pred[:5]

In [ ]:
# randomly plot some mislabeled images
def print_mislabeled_images(class_names, test_images, test_labels, pred_labels):
    BOO = (test_labels == pred_labels)
    mislabeled_indices = np.where(BOO == 0)
    mislabeled_images = test_images[mislabeled_indices]
    mislabeled_labels = pred_labels[mislabeled_indices]

    title = "Some examples of mislabeled images by the classifier:"
    display_examples(class_names,  mislabeled_images, mislabeled_labels)

In [ ]:
# print_mislabeled_images(class_names, trainX, trainy, train)
print_mislabeled_images(class_names, testX, testy, pred)

In [ ]:
# calculate the confusion matrix
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(testy, pred)
CM

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(testy, pred, target_names=class_names))

In [ ]:
confusion_matrix(trainy, train)

In [ ]:
# plot the confusion matrix
# data = confusion_matrix(trainy, train)
data = confusion_matrix(testy, pred)
df_cm = pd.DataFrame(data, columns=np.unique(testy), index = np.unique(testy))
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
plt.figure(figsize = (12, 10))
sns.heatmap(df_cm, cmap="Blues", annot=True, fmt='d', annot_kws={"size": 19}) 